In [1]:
!pip install gensim


In [0]:
import os
import re
import json

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout
from keras.layers import SpatialDropout1D, Conv1D, MaxPooling1D
from gensim.models import Word2Vec
from keras.utils import np_utils
from keras.preprocessing import sequence
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
import seaborn as sns
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import model_from_json

pd.options.display.max_rows = 10
pd.set_option('display.max_columns', None)

from google.colab import files
import pickle


from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
from keras.utils import to_categorical

from keras.models import load_model


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))

except KeyError:
  print('TPU not found')

TPU not found


In [0]:
w2v_path = os.path.join("/content/gdrive/My Drive/", "Colab Notebooks/ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec.gz")
hh_data_path = os.path.join("/content/gdrive/My Drive/", "Colab Notebooks/hh_dataset.csv")
jooble_data_path = os.path.join("/content/gdrive/My Drive/", "Colab Notebooks/by_jobs.csv")
weights_dir = os.path.join("/content/gdrive/My Drive/", "Colab Notebooks/weights")

In [6]:
dataset_hh = pd.read_csv(hh_data_path, sep="\t")
dataset_hh = dataset_hh.loc[dataset_hh["lang_text"] == "russian"]
#dataset_hh.dropna(inplace=True)
#dataset_hh = dataset_hh.loc[isinstance(dataset_hh["responsibility"], str)]
#dataset_hh.drop(dataset_hh.index[dataset_hh["lang_text"] == "english"], inplace=True)
dataset_hh.reset_index(drop=True, inplace=True)
print(dataset_hh.info())
#dataset_hh.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25223 entries, 0 to 25222
Data columns (total 26 columns):
id                            25223 non-null int64
lang_text                     25223 non-null object
lang_title                    25223 non-null object
profarea_names                25223 non-null object
requirement                   24322 non-null object
requirement_lemmas            24321 non-null object
requirement_lemmas_tags       24321 non-null object
requirement_norm              24321 non-null object
requirement_tokens            24321 non-null object
responsibility                23890 non-null object
responsibility_lemmas         23889 non-null object
responsibility_lemmas_tags    23889 non-null object
responsibility_norm           23889 non-null object
responsibility_tokens         23889 non-null object
specializations               25223 non-null object
text                          25223 non-null object
text_lemmas                   25223 non-null object
text_lem

In [82]:
dataset_hh.drop_duplicates(["text"], inplace=True)
print("Final size of dataset =", len(dataset_hh))
dataset_hh.reset_index(drop=True, inplace=True)
dataset_hh.info()

Final size of dataset = 17250
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17250 entries, 0 to 17249
Data columns (total 26 columns):
id                            17250 non-null int64
lang_text                     17250 non-null object
lang_title                    17250 non-null object
profarea_names                17250 non-null object
requirement                   16763 non-null object
requirement_lemmas            16762 non-null object
requirement_lemmas_tags       16762 non-null object
requirement_norm              16762 non-null object
requirement_tokens            16762 non-null object
responsibility                16481 non-null object
responsibility_lemmas         16480 non-null object
responsibility_lemmas_tags    16480 non-null object
responsibility_norm           16480 non-null object
responsibility_tokens         16480 non-null object
specializations               17250 non-null object
text                          17250 non-null object
text_lemmas                   

In [0]:
#dataset_hh.to_csv(os.path.join(weights_dir, "hh_dataset_all_uniq_text.csv"), sep='\t', header=True, index=None)

In [0]:
#files.download("hh_dataset_all_uniq_text.csv")

In [8]:
%%time
vectorizer = KeyedVectors.load_word2vec_format(w2v_path, binary=False)

CPU times: user 1min 56s, sys: 787 ms, total: 1min 57s
Wall time: 2min


### Prepare dataset

In [0]:
def get_mask(text, req):
    
    text = text.split(" ")
    req = req.split(" ")
    mask = np.zeros(len(text))
    
    i = 0
    while i  <= (len(text)):      
                   
        if (req == text[i: i + len(req)]):
            for j in range(len(req)):
                mask[i + j] = 1
            i += 1
            break

        i += 1
    return mask 

In [0]:
def get_training_sample(model, text):
    """
    Gets 1 sample of training data
    
    :param  model text:
    :return str text:
    """
    tagged_list = text.split(" ")

    vec_list = []
    
    #converting word2vec 
    for word in tagged_list:
        try:
            vec_list.append(model[word])
        except:
            #print("Word " + word + " isn't in vocab. Embeding as zeros")
            vec_list.append(np.zeros(300))
    return vec_list
  

In [11]:
lens = []
total_len = 0
for i, raw in dataset_hh.iterrows():
  doc_len = len(raw["text_lemmas"].split(" "))
  total_len += doc_len
  lens.append(doc_len)
  
mean_len = total_len / len( dataset_hh)
print("Mean text len =", mean_len)
  
print(np.mean(lens))
print(np.std(lens))

Mean text len = 111.80434782608695
111.80434782608695
65.93662072104469


In [0]:
#text = dataset_hh.loc[0, "text_lemmas_tags"]
#print(text)
#sub_text = dataset_hh.loc[0, "requirement_lemmas_tags"]
#print(sub_text)

In [0]:
#index = 0
#text = dataset_hh.loc[index, "text_lemmas"]
#sub_text = dataset_hh.loc[index, "requirement_lemmas"]
#sub_text2 = dataset_hh.loc[index, "responsibility_lemmas"]
#
#y1 = get_mask(text, sub_text)
#y2 = get_mask(text, sub_text2)
#y = np.logical_or(y1, y2).astype(int)
#
#x = get_training_sample(vectorizer, text)

In [0]:
#print(len(x))
#print(len(y))
#print(y)
#x = sequence.pad_sequences([x], maxlen=100, dtype='float', padding="post", truncating="post")
#y = sequence.pad_sequences([list(y)], maxlen=100, dtype='float', padding="post", truncating="post")
#print()
#print(y)
#print()
#print(len(x[0]))
#print(len(y[0]))

In [0]:
#x[0][99]

In [0]:
#print(sub_text)
#print(sub_text2)
#print(text)

In [0]:
def get_one_sample(row, max_len=200):
  if not isinstance(row["requirement_norm"], str) or not isinstance(row["responsibility_norm"], str) :
    print("Try another document")
    return None
  text = row["text_lemmas_tags"]
  x_sample = get_training_sample(vectorizer, text)

  text = row["text_lemmas"]
  sub_text = row["requirement_lemmas"]
  sub_text2 = row["responsibility_lemmas"]
  
  y1 = get_mask(text, sub_text)
  y2 = get_mask(text, sub_text2) 
  y_sample = np.logical_or(y1, y2).astype(int)
  
  assert(len(x_sample) == len(y_sample))
  
  x_sample = sequence.pad_sequences([x_sample], maxlen=max_len, dtype='float', padding="post", truncating="post")
  y_sample = sequence.pad_sequences([list(y_sample)], maxlen=max_len, dtype='float', padding="post", truncating="post")
  
  y_sample = np.array(y_sample)
  y_sample = np.reshape(y_sample, (1, -1, 1))
  x_sample = np.array(x_sample)
  
  y_new = np.zeros((y_sample.shape[0], y_sample.shape[1], 2))
  for j in range(y_sample.shape[1]):
    y_new[0, j] = to_categorical(y_sample[0, j, 0], num_classes=2)
    
  y_sample = y_new
  #print("x shape =", x_sample.shape, " y shape =", y_sample.shape)
  
  return x_sample, y_sample
  

In [0]:
x, y = get_one_sample(dataset_hh.loc[0], max_len=200)

In [14]:
%%time
X = []
Y = []
max_len_real = 0
max_len = 200
n_samples = 10_709
# 10_000: 12_000 - for text
for i, row in dataset_hh.loc[: n_samples].iterrows():
  if not isinstance(row["requirement_norm"], str) or not isinstance(row["responsibility_norm"], str) :
    continue
  text = row["text_lemmas_tags"]
  x_sample = get_training_sample(vectorizer, text)

  text = row["text_lemmas"]
  sub_text = row["requirement_lemmas"]
  sub_text2 = row["responsibility_lemmas"]
  
  y1 = get_mask(text, sub_text)
  y2 = get_mask(text, sub_text2) 
  y_sample = np.logical_or(y1, y2).astype(int)
  
  Y.append(y_sample)
  X.append(x_sample)
  
  if (len(x_sample) != len(y_sample)):
    print(len(x_sample))
    print(len(y_sample))
    print(len(row["text_normalized"].split(" ")))
    print(y_sample)
    print(i)
  assert(len(x_sample) == len(y_sample))
  
  if len(y_sample) > max_len_real:
    max_len_real = len(y_sample)
    
X = sequence.pad_sequences(X, maxlen=max_len, dtype='float', padding="post", truncating="post")
Y = sequence.pad_sequences(Y, maxlen=max_len, dtype='float', padding="post", truncating="post")
       
Y = np.array(Y)
Y = np.reshape(Y, (Y.shape[0], Y.shape[1], -1))
X = np.array(X)
print("Max len real=", max_len_real)
print("Max len to reshape =", max_len)


Max len real= 814
Max len to reshape = 200
CPU times: user 9.33 s, sys: 5.5 s, total: 14.8 s
Wall time: 14.8 s


In [0]:
#print(Y[index])

In [15]:
print(X.shape)
print(Y.shape)

(10000, 200, 300)
(10000, 200, 1)


### seq2seq model

In [16]:
%%time
Y_new = np.zeros((Y.shape[0], Y.shape[1], 2))
for i in range(Y.shape[0]):
  for j in range(Y.shape[1]):
    Y_new[i, j] = to_categorical(Y[i, j, 0], num_classes=2)
    
Y = Y_new
print(Y.shape)

(10000, 200, 2)
CPU times: user 18 s, sys: 10.2 ms, total: 18 s
Wall time: 18 s


In [17]:
%%time
encoder_input_data = X
decoder_input_data = Y

decoder_target_data = np.zeros(Y.shape)

for i in range(Y.shape[0]):
  for j in range(0, Y.shape[1]-1):
    decoder_target_data[i, j] =  Y[i, j + 1]

CPU times: user 1.74 s, sys: 3.06 ms, total: 1.75 s
Wall time: 1.75 s


In [0]:
#for i in range(0, 30):
#    print('Y:', np.argmax(Y[index, i]), 'Target', np.argmax(decoder_target_data[index, i]))

In [0]:
max_len = 200

num_encoder_tokens = 300
latent_dim = 400

num_decoder_tokens = 2
max_decoder_seq_length = max_len

batch_size = 64
epochs = 40

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [0]:
callbacks = [ModelCheckpoint(os.path.join(weights_dir, 'seq2seq_2.{epoch:02d}-{acc:.2f}.hdf5'))]

In [0]:

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = Bidirectional(LSTM(latent_dim // 2, return_state=True))

#a = encoder(encoder_inputs)
#print(len(a))
#encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
state_h = Concatenate(axis=1)([forward_h, backward_h])
state_c = Concatenate(axis=1)([forward_c, backward_c])

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
#decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [0]:
#model = load_model(os.path.join(weights_dir, 'seq2seq_2.33-0.99.hdf5'))

In [0]:
#model.save_weights(os.path.join(weights_dir, 'seq2seq_2.33-0.99_weights.h5'))

In [0]:
model.load_weights(os.path.join(weights_dir, 'seq2seq_2.33-0.99_weights.h5'))

In [0]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', loss_weights=[[0.1, 1000_000]], metrics=['accuracy'])


In [42]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 300)    0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 400), (None, 801600      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 2)      0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 400)          0           bidirectional_1[0][1]            
                                                                 bidirectional_1[0][3]            
__________

In [54]:
%%time
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          #validation_split=0.1,
          callbacks=callbacks,
          verbose=1)
#  class_weight={0: 1.0, 1: 1000}

Epoch 1/40
10000/10000 [==============================] - 221s 22ms/step - loss: 15733.7637 - acc: 0.9891


/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_3 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'concatenate_1_2/concat:0' shape=(?, 400) dtype=float32>, <tf.Tensor 'concatenate_2_2/concat:0' shape=(?, 400) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/40
10000/10000 [==============================] - 220s 22ms/step - loss: 16634.4121 - acc: 0.9888
Epoch 3/40
10000/10000 [==============================] - 220s 22ms/step - loss: 15873.1699 - acc: 0.9890
Epoch 4/40
10000/10000 [==============================] - 217s 22ms/step - loss: 15373.4414 - acc: 0.9892
Epoch 5/40
10000/10000 [==============================] - 221s 22ms/step - loss: 15024.5166 - acc: 0.9894
Epoch 6/40
10000/10000 [==============================] - 217s 22ms/step - loss: 14707.8164 - acc: 0.9895
Epoch 7/40
10000/10000 [==============================] - 216s 22ms/step - loss: 14570.2266 - acc: 0.9896
Epoch 8/40
10000/10000 [==============================] - 219s 22ms/step - loss: 14304.5293 - acc: 0.9898
Epoch 9/40
10000/10000 [==============================] - 218s 22ms/step - loss: 14106.6826 - acc: 0.9899
Epoch 10/40
10000/10000 [==============================] - 217s 22ms/step - loss: 15381.5117 - acc: 0.9894
Epoch 11/40
10000/10000 [====================

In [44]:
model.layers

In [0]:

#encoder_inputs = model.layers[0]
#encoder_states = model.layers[3: 5]
#decoder_inputs = model.layers[2]

#decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)



In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        """sampled_char = reverse_target_char_index[sampled_token_index]"""
        decoded_sentence.append(sampled_token_index)

        # Exit condition: either hit max length
        # or find stop character.
        if (len(decoded_sentence) == max_decoder_seq_length) or \
        (len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Save model

In [57]:
with open(os.path.join(weights_dir, 'encoder_model.json'), 'w', encoding='utf8') as f:
    f.write(encoder_model.to_json())
encoder_model.save_weights(os.path.join(weights_dir,'encoder_model_weights.h5'))

with open(os.path.join(weights_dir,'decoder_model.json'), 'w', encoding='utf8') as f:
    f.write(decoder_model.to_json())
decoder_model.save_weights(os.path.join(weights_dir, 'decoder_model_weights.h5'))
        

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_3 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'input_5:0' shape=(?, 400) dtype=float32>, <tf.Tensor 'input_6:0' shape=(?, 400) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [0]:
del X
del Y

In [0]:
files.download("encoder_model.json")
files.download("decoder_model.json")
files.download("encoder_model_weights.h5")
files.download("decoder_model_weights.h5")

### Load model from file to test

In [0]:
#files.upload()


{}

In [0]:
#def load_model(model_filename, model_weights_filename):
#    with open(model_filename, 'r', encoding='utf8') as f:
#        model = model_from_json(f.read())
#    model.load_weights(model_weights_filename)
#    return model
#
#encoder_model = load_model('encoder_model.json', 'encoder_model_weights.h5')
#decoder_model = load_model('decoder_model.json', 'decoder_model_weights.h5')

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        """sampled_char = reverse_target_char_index[sampled_token_index]"""
        decoded_sentence.append(sampled_token_index)

        # Exit condition: either hit max length
        # or find stop character.
        if (len(decoded_sentence) == max_decoder_seq_length) or \
        (len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
index = 12001

In [95]:
%%time
x, y = get_one_sample(dataset_hh.loc[index])

CPU times: user 4.71 ms, sys: 939 µs, total: 5.65 ms
Wall time: 7.04 ms


In [96]:
%%time
predict = decode_sequence(x)

CPU times: user 870 ms, sys: 140 ms, total: 1.01 s
Wall time: 812 ms


In [97]:
print(dataset_hh.loc[index, "title"])
print()
print(dataset_hh.loc[index, "requirement_lemmas"])
print(dataset_hh.loc[index, "responsibility_lemmas"])
print(dataset_hh.loc[index, "text_lemmas"])

Курьер

пунктуальность четкий выполнение ставить задача опыт объяснить весь научить желание иметь стабильный заработок
работа закрепленный участок выбор оформление документ заполнение накладной
ооо премьеравто приглашать сотрудничество пеший курьер постоянный основа временный заработок выплата возможный ежедневный еженедельный условие работа закреплять участок выбор оплата проезд питание требование пунктуальность четкий выполнение ставить задача опыт объяснить весь научить желание иметь стабильный заработок обязанность доставка корреспонденция возможный зависимость закреплять территория оформление документ заполнение накладной прием оплата доставка прямой работодатель набирать большой штат сотрудник самый надежный человек готовый работать постоянный основа труд поощрять премия различный бонус


In [98]:
words = dataset_hh.loc[index, "text_lemmas"].split(" ")
for i in range(0, 100):
  if len(words) > i:
    word = words[i]
  else:
    word = "END"
  print(i, 'Expected:', np.argmax(y[0, i]), 'Predicted', predict[i], word)

0 Expected: 0 Predicted 0 ооо
1 Expected: 0 Predicted 0 премьеравто
2 Expected: 0 Predicted 0 приглашать
3 Expected: 0 Predicted 0 сотрудничество
4 Expected: 0 Predicted 0 пеший
5 Expected: 0 Predicted 0 курьер
6 Expected: 0 Predicted 0 постоянный
7 Expected: 0 Predicted 0 основа
8 Expected: 0 Predicted 0 временный
9 Expected: 0 Predicted 0 заработок
10 Expected: 0 Predicted 0 выплата
11 Expected: 0 Predicted 0 возможный
12 Expected: 0 Predicted 0 ежедневный
13 Expected: 0 Predicted 0 еженедельный
14 Expected: 0 Predicted 0 условие
15 Expected: 0 Predicted 0 работа
16 Expected: 0 Predicted 0 закреплять
17 Expected: 0 Predicted 0 участок
18 Expected: 0 Predicted 0 выбор
19 Expected: 0 Predicted 0 оплата
20 Expected: 0 Predicted 0 проезд
21 Expected: 0 Predicted 0 питание
22 Expected: 0 Predicted 1 требование
23 Expected: 1 Predicted 1 пунктуальность
24 Expected: 1 Predicted 1 четкий
25 Expected: 1 Predicted 1 выполнение
26 Expected: 1 Predicted 1 ставить
27 Expected: 1 Predicted 1 задач

### Text on jooble data

In [99]:
dataset_jooble = pd.read_csv(jooble_data_path, sep="\t")
dataset_jooble = dataset_jooble.loc[dataset_jooble["lang_text"] == "russian"]
#dataset_hh.dropna(inplace=True)
#dataset_hh = dataset_hh.loc[isinstance(dataset_hh["responsibility"], str)]
#dataset_hh.drop(dataset_hh.index[dataset_hh["lang_text"] == "english"], inplace=True)
dataset_jooble.reset_index(drop=True, inplace=True)
print(dataset_jooble.info())
#dataset_jooble.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9237 entries, 0 to 9236
Data columns (total 14 columns):
id                   9237 non-null int64
title                9237 non-null object
lang_title           9237 non-null object
title_normalized     8285 non-null object
title_lemmas         8285 non-null object
title_lemmas_tags    8285 non-null object
title_tokens         8285 non-null object
text                 9237 non-null object
lang_text            9237 non-null object
text_normalized      9236 non-null object
text_lemmas          9236 non-null object
text_lemmas_tags     9236 non-null object
text_tokens          9236 non-null object
url                  9237 non-null object
dtypes: int64(1), object(13)
memory usage: 1010.4+ KB
None


In [0]:
maxlen = 200
def get_sample_nonlabeled(row):

  text = row["text_lemmas_tags"]
  x_sample = get_training_sample(vectorizer, text)
  x_sample = sequence.pad_sequences([x_sample], maxlen=max_len, dtype='float', padding="post", truncating="post")
  x_sample = np.array(x_sample)
  
  #print("x shape =", x_sample.shape)
  
  return x_sample
  

In [101]:
%%time
indexes_labeled = []
n_test = 100

for i, row in dataset_jooble.loc[: n_test].iterrows():
  x = get_sample_nonlabeled(dataset_jooble.loc[i])
  predict = decode_sequence(x)
  
  if np.sum(predict) > 0:
    indexes_labeled.append(i)
    
  if i % 100 == 0:
    print(i)
  
print()
print("Total:")
print(len(indexes_labeled))

0
100

Total:
13
CPU times: user 1min 27s, sys: 12.7 s, total: 1min 40s
Wall time: 1min 19s


80 epochs
Total:
15

In [102]:
print(indexes_labeled)

[8, 9, 18, 27, 36, 41, 57, 61, 65, 68, 69, 85, 97]


80 epoch
[8, 9, 18, 27, 36, 57, 61, 63, 68, 70, 71, 85, 87, 97, 100]

new(best):
[8, 9, 18, 27, 36, 41, 57, 61, 65, 68, 69, 85, 97]

In [118]:
%%time
index = 65

x = get_sample_nonlabeled(dataset_jooble.loc[index])

predict = decode_sequence(x)

print(np.sum(predict))

print(dataset_jooble.loc[index, "title"])
print()
print(dataset_jooble.loc[index, "text"])

14
Швея трикотажных и корсетных изделий с опытом работы




















CPU times: user 824 ms, sys: 155 ms, total: 979 ms
Wall time: 809 ms


In [119]:
print(np.sum(predict))
words = dataset_jooble.loc[index, "text_lemmas"].split(" ")
for i in range(0, 50):
  if len(words) > i:
    word = words[i]
  else:
    word = "END"
  print(i, 'Predicted', predict[i], " - ", word)
  

14
0 Predicted 0  -  производитель
1 Predicted 0  -  женский
2 Predicted 0  -  белье
3 Predicted 0  -  хлопок
4 Predicted 0  -  ооо
5 Predicted 0  -  медеястиль
6 Predicted 0  -  искать
7 Predicted 0  -  шветь
8 Predicted 0  -  опыт
9 Predicted 0  -  работа
10 Predicted 0  -  город
11 Predicted 0  -  минскать
12 Predicted 0  -  характер
13 Predicted 0  -  работа
14 Predicted 0  -  территория
15 Predicted 0  -  работодатель
16 Predicted 0  -  график
17 Predicted 0  -  работа
18 Predicted 0  -  фиксированный
19 Predicted 0  -  сменный
20 Predicted 0  -  занятость
21 Predicted 0  -  полный
22 Predicted 0  -  требование
23 Predicted 1  -  кандидат
24 Predicted 1  -  опыт
25 Predicted 1  -  работа
26 Predicted 1  -  год
27 Predicted 1  -  образование
28 Predicted 1  -  иметь
29 Predicted 1  -  значение
30 Predicted 1  -  студент
31 Predicted 1  -  опыт
32 Predicted 1  -  работа
33 Predicted 1  -  швеий
34 Predicted 1  -  трикотажный
35 Predicted 1  -  корсетный
36 Predicted 1  -  изделие
37

### Test

In [70]:
%%time
X = []
Y = []
max_len_real = 0
max_len = 200
n_samples = 10_709
# 10_000: 12_000 - for text
for i, row in dataset_hh.loc[10_000: 13_000].iterrows():
  if not isinstance(row["requirement_norm"], str) or not isinstance(row["responsibility_norm"], str) :
    continue
  text = row["text_lemmas_tags"]
  x_sample = get_training_sample(vectorizer, text)

  text = row["text_lemmas"]
  sub_text = row["requirement_lemmas"]
  sub_text2 = row["responsibility_lemmas"]
  
  y1 = get_mask(text, sub_text)
  y2 = get_mask(text, sub_text2) 
  y_sample = np.logical_or(y1, y2).astype(int)
  
  Y.append(y_sample)
  X.append(x_sample)
  
  if (len(x_sample) != len(y_sample)):
    print(len(x_sample))
    print(len(y_sample))
    print(len(row["text_normalized"].split(" ")))
    print(y_sample)
    print(i)
  assert(len(x_sample) == len(y_sample))
  
  if len(y_sample) > max_len_real:
    max_len_real = len(y_sample)
    
X = sequence.pad_sequences(X, maxlen=max_len, dtype='float', padding="post", truncating="post")
Y = sequence.pad_sequences(Y, maxlen=max_len, dtype='float', padding="post", truncating="post")
       
Y = np.array(Y)
Y = np.reshape(Y, (Y.shape[0], Y.shape[1], -1))
X = np.array(X)
print("Max len real=", max_len_real)
print("Max len to reshape =", max_len)

Max len real= 819
Max len to reshape = 200
CPU times: user 2.19 s, sys: 8.63 s, total: 10.8 s
Wall time: 10.8 s


In [71]:
%%time
#X.shape[0]
Y_predict = []
for i in range(X.shape[0]):
  predict = decode_sequence(X[i].reshape((1, max_len, 300)))
  Y_predict.append(predict)
  
  if i % 100 == 0:
    print(i)
  

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
CPU times: user 38min 26s, sys: 5min 34s, total: 44min
Wall time: 34min 58s


In [72]:
Y_predict = np.array(Y_predict)
Y_predict.shape

(2788, 200)

In [73]:
Y.shape

(2788, 200, 1)

In [75]:
np.sum(Y[:, :, 0])

58027.0

In [76]:
np.sum(Y_predict)

62913

In [0]:
from sklearn import metrics

In [77]:
tp = 0

for i in range(Y.shape[0]):
  for j in range(Y.shape[1]):
    if Y_predict[i, j] == Y[i, j]:
      tp += 1
      
accuracy = tp / (Y.shape[0] * Y.shape[1]) * 100


print("Accuracy =", accuracy)
  

Accuracy = 94.28156384505021


60 epochs
Accuracy = 97.1189866939611

In [78]:
f1_score = metrics.f1_score(Y.reshape(-1), Y_predict.reshape(-1))
print("F1-score =", f1_score)

F1-score = 0.7363486026128658


In [79]:
f1_score = metrics.f1_score(Y.reshape(-1), Y_predict.reshape(-1), average='weighted')
print("F1-score =", f1_score)

F1-score = 0.9438302603171191


F1-score = 0.8140268578318826

In [80]:
print(metrics.classification_report(Y.reshape(-1), Y_predict.reshape(-1)))
print("Confussion matrix: \n", metrics.confusion_matrix(Y.reshape(-1), Y_predict.reshape(-1), labels=[0, 1]))

             precision    recall  f1-score   support

        0.0       0.97      0.96      0.97    499573
        1.0       0.71      0.77      0.74     58027

avg / total       0.95      0.94      0.94    557600

Confussion matrix: 
 [[481187  18386]
 [ 13500  44527]]


80 epochs

precision    recall  f1-score   support

        0.0       0.98      0.99      0.98    359547
        1.0       0.84      0.79      0.81     31253

avg / total       0.97      0.97      0.97    390800

Confussion matrix: 

 [[354900   4647]
 
 [  6612  24641]]

In [0]:
#count = 0 
#for i in range(Y_predict.shape[0]):
#  if np.sum(Y_predict[i, :]) != 0:
#    print(" I =", i, "____________")
#    print(Y_predict[i])
#    count += 1
#print("total =", count)

In [0]:
files.download("encoder_model.json")
files.download("decoder_model.json")
files.download("encoder_model_weights.h5")
files.download("decoder_model_weights.h5")
